In [2]:
library(data.table)
library(limma)
library(dplyr)
library(edgeR)
library(tibble)
library(biomaRt)
library(tidyr)
library(stringr)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
## Import gene names

In [4]:
ensembl = useEnsembl(biomart="ensembl", dataset="rnorvegicus_gene_ensembl", version=100)

In [5]:
gene_annot <- getBM(mart=ensembl, attributes=c('ensembl_gene_id', 'external_gene_name'))

In [6]:
# Set up I/O

In [7]:
base_dir <- "../"
species <- 'rn'
save_dir <- file.path(base_dir, 'DE_and_abundance')
meta_path <- file.path(base_dir, "meta_data", "meta_data.csv")
counts_path <- file.path(base_dir, "feature_counts", paste0("trimmed_gene_counts_", species, '.csv'))
annot_path <- file.path(base_dir, "feature_counts", paste0("trimmed_FC_gene_annot_", species, '.csv'))

In [8]:
# Read in counts, annotations, and meta data 

In [9]:
FC_annot <- read.csv(annot_path, row.names=1)
## Select only gene id and length. Note that dplyr will drop row names upon select
## but edgeR expects gene id in row names, so convert gene_id to column, then back, 
## to row names after selecting
FC_annot <- rownames_to_column(FC_annot, var='gene_id')
FC_annot <- dplyr::select(FC_annot, 'gene_id', 'Length')
FC_annot <- column_to_rownames(FC_annot, var='gene_id')

Warning message in file(file, "rt"):
“cannot open file '..//feature_counts/trimmed_FC_gene_annot_rn.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


In [ ]:
meta <- read.csv(meta_path)

In [ ]:
counts <- read.csv(counts_path, row.names=1)

In [ ]:
## Change meta sample names to be acceptable variable names

In [ ]:
meta$RatID <- factor(meta$RatID)

In [ ]:
## Take counts for only samples in meta table

In [ ]:
counts <- dplyr::select(counts, meta$Sample)

In [ ]:
## Get differential gene expression list for use in limma

In [ ]:
y <- DGEList(counts, genes=FC_annot)

In [ ]:
## Define the linear model and get the design matrix

In [ ]:
design <- model.matrix(~ 0 + Condition, data=meta)

In [ ]:
## Filter the data to remove low expressed genes

In [ ]:
keep <- filterByExpr(y, design, min.count=10, min.total.count=15)
y <- y[keep, keep.lib.sizes=FALSE]

In [ ]:
## Calculate library normalization factors 

In [ ]:
y <- calcNormFactors(y)

In [ ]:
v <- voom(y,design, plot=TRUE)

In [ ]:
## Fit the linear model and perform empirical Bayes smoothing

In [ ]:
fit <- lmFit(v,design)
fit <- eBayes(fit)

In [ ]:
# Get contrasts as all pairs of conditions
conditions <- colnames(design)
contr <- c()
#print(conditions)
for (i in 1:(length(conditions)-1)){
    for (j in (i+1):length(conditions)){
     contrast <- paste0(conditions[i], '-', conditions[j])
        contr <- c(contr, contrast)
    }
}

contrast.matrix <- makeContrasts(contrasts=contr, levels=design)

In [26]:
## Fit coefficients for contrasts

In [27]:
fit2 <- contrasts.fit(fit, contrast.matrix)
fit2 <- eBayes(fit2)

In [28]:
############################################################
# Prepare DE top tables
############################################################

In [29]:
## Get abundance table with more readable samples names for combining with top table

In [31]:
voom_counts <- as.data.frame(v$E)
abundance <- 2.**(copy(voom_counts))
abundance <- setnames(abundance, old=meta$Sample, new=meta$Label)
abundance <- rownames_to_column(abundance, var='ensembl_gene_id')

In [32]:
## Get RPKM and convert to TPM

In [33]:
RPKM <- copy(rpkm(y))
TPM <- sweep(RPKM, 2, colSums(RPKM), '/')*(1.e6)
TPM <- setnames(as.data.frame(TPM), old=meta$Sample, new=meta$Label)
TPM <- rownames_to_column(TPM, var='ensembl_gene_id')

In [34]:
## Get top table with logFC and FDR

In [35]:
## Generate empty list to store top tables in 
tt_list <- list()
i <- 1
for (contrast in contr){
    top_table <- topTable(fit2, coef=contrast, number=Inf, sort.by="t")
    top_table <- rownames_to_column(top_table, var='ensembl_gene_id')
    top_table <- left_join(top_table, gene_annot, by='ensembl_gene_id')
    top_table <- dplyr::select(top_table, ensembl_gene_id, external_gene_name, everything())
    tt_list[[i]] <- top_table
    top_table_with_abundance <- left_join(top_table, abundance, by='ensembl_gene_id')
    top_table_with_TPM <- left_join(top_table, TPM, by='ensembl_gene_id')
    contrast_label <- str_remove_all(contrast, 'Condition')
    
    # Save top table (no abundance)
    filename <- paste0('toptable_', species, '_', contrast_label, '.csv')
    savepath <- file.path(save_dir, 'top_tables', filename)
    write.csv(top_table, savepath, row.names=FALSE)
    
    # Save top table with CPM abundance
    filename <- paste0('toptable_and_CPM_', species, '_', contrast_label, '.csv')
    savepath <- file.path(save_dir, 'top_tables_with_CPM_abundance', filename)
    write.csv(top_table_with_abundance, savepath, row.names=FALSE)
    
    # Save top table with CPM abundance
    filename <- paste0('toptable_and_TPM_', species, '_', contrast_label, '.csv')
    savepath <- file.path(save_dir, 'top_tables_with_TPM_abundance', filename)
    write.csv(top_table_with_TPM, savepath, row.names=FALSE)
   
    i <- i + 1
}

In [36]:
################################################
# Prepare a table that combines all DE results
################################################

In [37]:
# replace contrasts with more readable labels

In [38]:
contr_label = str_remove_all(contr, 'Condition')
contr_label = paste0('_', contr_label)

In [39]:
# Join the tables

In [40]:
top_table_all <- tt_list[[1]]
for (i in 1:(length(contr_label)-1)){
    if (i < length(contr_label) -1){ 
        top_table_all <- left_join(top_table_all, tt_list[[i+1]], 
                           by=c('ensembl_gene_id', 'external_gene_name', 'Length'), 
                           suffix = c(contr_label[i], ''))
        } else{
            top_table_all <- left_join(top_table_all, tt_list[[i+1]], 
                               by=c('ensembl_gene_id', 'external_gene_name', 'Length'), 
                               suffix = c(contr_label[i], contr_label[i+1]))
        } 
            
    
}
top_table_all_CPM <- left_join(top_table_all, abundance, by='ensembl_gene_id')
top_table_all_TPM <- left_join(top_table_all, TPM, by='ensembl_gene_id')


In [41]:
# Save the top tables

In [42]:
filename <- paste0('top_table_all_comparisons_',species, '.csv')
savepath <- file.path(save_dir, 'top_tables', filename)
write.csv(top_table_all, savepath, row.names=FALSE)

In [43]:
filename <- paste0('top_table_all_comparisons_', species, 'CPM.csv')
savepath <- file.path(save_dir, 'top_tables_with_CPM_abundance', filename)
write.csv(top_table_all_CPM, savepath, row.names=FALSE)

In [44]:
filename <- paste0('top_table_all_comparisons_', species,'TPM.csv')
savepath <- file.path(save_dir, 'top_tables_with_TPM_abundance', filename)
write.csv(top_table_all_TPM, savepath, row.names=FALSE)

In [45]:
##################################
## Create tidy TPM and counts
#################################

In [46]:
## Cast the TPM table into tidy format and add sample info

In [47]:
RPKM <- copy(rpkm(y))
TPM2 <- sweep(RPKM, 2, colSums(RPKM), '/')*(1.e6)
TPM2 <- rownames_to_column(as.data.frame(TPM2), var='ensembl_gene_id')

In [48]:
TPM.tidy <- pivot_longer(TPM2, -ensembl_gene_id, names_to='Sample', values_to='TPM')
TPM.tidy <- left_join(TPM.tidy, meta, by='Sample')
TPM.tidy <- left_join(TPM.tidy, gene_annot, by='ensembl_gene_id')
TPM.tidy <- dplyr::select(TPM.tidy, ensembl_gene_id, external_gene_name, colnames(meta), everything())

In [49]:
filename <- paste0('TPM_', species, '.csv')
savefile <- file.path(save_dir, 'tidy_abundance_tables', filename)
write.csv(TPM.tidy, savefile)

In [50]:
## Get the voom normalized counts per million

In [51]:
voom_cpm <- v$E
voom_cpm2 <- 2**voom_cpm
voom_cpm3 <- rownames_to_column(as.data.frame(voom_cpm2), var='ensembl_gene_id')
voom_cpm_tidy <- pivot_longer(voom_cpm3, -ensembl_gene_id, names_to='Sample', values_to='CPM')
voom_cpm_tidy <- left_join(voom_cpm_tidy, meta, by='Sample')
voom_cpm_tidy <- left_join(voom_cpm_tidy, gene_annot, by='ensembl_gene_id')
voom_cpm_tidy <- dplyr::select(voom_cpm_tidy, ensembl_gene_id, external_gene_name, colnames(meta), everything())
filename <- paste0('cpm_voom_tidy_',species, '.csv')
voom_cpm_save_file <- file.path(save_dir, 'tidy_abundance_tables', filename)
write.csv(voom_cpm_tidy, voom_cpm_save_file, row.names=FALSE)

In [52]:
## Get tidy raw counts`

In [53]:
counts2 <- rownames_to_column(counts, var='ensembl_gene_id')
genes <- gene_annot['ensembl_gene_id']
counts2 <- dplyr::filter(counts2, ensembl_gene_id %in% gene_annot$ensembl_gene_id)
counts.tidy <- pivot_longer(counts2, -ensembl_gene_id, names_to='Sample', values_to='Counts')
counts.tidy <- left_join(counts.tidy, meta, by='Sample')
counts.tidy <- left_join(counts.tidy, gene_annot, by='ensembl_gene_id')
counts.tidy <- dplyr::select(counts.tidy, ensembl_gene_id, external_gene_name, colnames(meta), everything())

In [54]:
filename <- paste0('Unfiltered_counts_tidy_', species, '.csv')
counts_save_file <- file.path(save_dir, 'tidy_abundance_tables', filename)
write.csv(counts.tidy, counts_save_file, row.names=FALSE)

In [55]:
## Merge 

In [56]:
TPM_counts_tidy <- left_join(TPM.tidy, counts.tidy, by=append(c('ensembl_gene_id', 'external_gene_name'),colnames(meta)))

In [57]:
TPM_cpm_tidy <- left_join(TPM.tidy, voom_cpm_tidy, by=append(c('ensembl_gene_id', 'external_gene_name'),colnames(meta)))

In [58]:
TPM_counts_cpm <- left_join(TPM_cpm_tidy, counts.tidy, by=append(c('ensembl_gene_id', 'external_gene_name'),colnames(meta)))
TPM_counts_cpm <- dplyr::select(TPM_counts_cpm, ensembl_gene_id, external_gene_name, colnames(meta), everything())

In [59]:
filename <- paste0('TPM_CPM_counts_', species, '.csv')
tpm_cpm_cts_save_file <- file.path(save_dir, 'tidy_abundance_tables', filename)
write.csv(TPM_counts_cpm, tpm_cpm_cts_save_file, row.names=FALSE)